In [1]:
# By Justin Johnson https://github.com/jcjohnson/pytorch-examples/blob/master/nn/dynamic_net.py

import random
import torch
from torch.autograd import Variable

"""
To showcase the power of PyTorch dynamic graphs, we will implement a very strange
model: a fully-connected ReLU network that on each forward pass randomly chooses
a number between 1 and 4 and has that many hidden layers, reusing the same
weights multiple times to compute the innermost hidden layers.
"""

class DynamicNet(torch.nn.Module):
  def __init__(self, D_in, H, D_out):
    """
    In the constructor we construct three nn.Linear instances that we will use
    in the forward pass.
    """
    super(DynamicNet, self).__init__()
    self.input_linear = torch.nn.Linear(D_in, H)
    self.middle_linear = torch.nn.Linear(H, H)
    self.output_linear = torch.nn.Linear(H, D_out)

  def forward(self, x, verbose = False):
    """
    For the forward pass of the model, we randomly choose either 0, 1, 2, or 3
    and reuse the middle_linear Module that many times to compute hidden layer
    representations.
    Since each forward pass builds a dynamic computation graph, we can use normal
    Python control-flow operators like loops or conditional statements when
    defining the forward pass of the model.
    Here we also see that it is perfectly safe to reuse the same Module many
    times when defining a computational graph. This is a big improvement from Lua
    Torch, where each Module could be used only once.
    """
    h_relu = self.input_linear(x).clamp(min=0)
    n_layers = random.randint(0, 3)
    if verbose:
        print("The number of layers for this run is", n_layers)
        # print(h_relu)
    for _ in range(n_layers):
        h_relu = self.middle_linear(h_relu).clamp(min=0)
        if verbose:
            pass
            # print(h_relu)
    y_pred = self.output_linear(h_relu)
    return y_pred




# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 10, 1

# Create random Tensors to hold inputs and outputs, and wrap them in Variables
x = Variable(torch.randn(N, D_in))
y = Variable(torch.randn(N, D_out), requires_grad=False)

# Construct our model by instantiating the class defined above
model = DynamicNet(D_in, H, D_out)

# Construct our loss function and an Optimizer. Training this strange model with
# vanilla stochastic gradient descent is tough, so we use momentum
criterion = torch.nn.MSELoss(size_average=False)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)
for t in range(500):
  # Forward pass: Compute predicted y by passing x to the model
  y_pred = model(x)

  # Compute and print loss
  loss = criterion(y_pred, y)
  print(t, loss.data[0])

  # Zero gradients, perform a backward pass, and update the weights.
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

(0, 68.55696105957031)
(1, 64.40031433105469)
(2, 57.4582405090332)
(3, 68.0625228881836)
(4, 67.96131896972656)
(5, 67.4065933227539)
(6, 67.52629089355469)
(7, 64.49079132080078)
(8, 31.04802131652832)
(9, 27.771982192993164)
(10, 66.30720520019531)
(11, 61.11796951293945)
(12, 59.83041763305664)
(13, 16.875253677368164)
(14, 14.834412574768066)
(15, 63.62442398071289)
(16, 53.64186096191406)
(17, 51.711700439453125)
(18, 65.10582733154297)
(19, 59.95439147949219)
(20, 45.11320495605469)
(21, 42.68074417114258)
(22, 40.06452941894531)
(23, 12.281163215637207)
(24, 51.818416595458984)
(25, 32.959800720214844)
(26, 47.32932662963867)
(27, 12.875962257385254)
(28, 11.506110191345215)
(29, 40.70511245727539)
(30, 55.15180969238281)
(31, 36.384178161621094)
(32, 5.563450336456299)
(33, 4.731485366821289)
(34, 3.8788135051727295)
(35, 21.744544982910156)
(36, 43.13826370239258)
(37, 41.036827087402344)
(38, 26.631256103515625)
(39, 18.915164947509766)
(40, 24.17453384399414)
(41, 22.682395

(361, 0.12505511939525604)
(362, 0.12213917821645737)
(363, 0.1546889692544937)
(364, 0.18771693110466003)
(365, 0.11050982773303986)
(366, 0.18495656549930573)
(367, 0.10939125716686249)
(368, 0.1789781153202057)
(369, 0.16344432532787323)
(370, 0.1720968782901764)
(371, 0.11583121865987778)
(372, 0.16257531940937042)
(373, 0.10569147765636444)
(374, 0.16535578668117523)
(375, 0.155373215675354)
(376, 0.16380517184734344)
(377, 0.16193357110023499)
(378, 0.13198710978031158)
(379, 0.1080806702375412)
(380, 0.10643331706523895)
(381, 0.15422247350215912)
(382, 0.10351108014583588)
(383, 0.159107968211174)
(384, 0.10121788084506989)
(385, 0.15437807142734528)
(386, 0.15408433973789215)
(387, 0.15222929418087006)
(388, 0.09942464530467987)
(389, 0.12104000151157379)
(390, 0.1467503309249878)
(391, 0.1176227480173111)
(392, 0.1447749137878418)
(393, 0.14344052970409393)
(394, 0.16800083220005035)
(395, 0.13989242911338806)
(396, 0.10323711484670639)
(397, 0.1639036238193512)
(398, 0.11775

In [2]:
model(x)[1:5]

Variable containing:
 2.7784
-0.5659
-1.1524
-0.6715
[torch.FloatTensor of size 4x1]

In [3]:
model(x)[1:5] # another run

Variable containing:
 2.7171
-0.5757
-1.1496
-0.6636
[torch.FloatTensor of size 4x1]

In [4]:
model(x)[1:5]

Variable containing:
 2.7219
-0.6037
-1.1652
-0.7280
[torch.FloatTensor of size 4x1]

Looks consistent! Let's now try to see what's happening inside

In [5]:
model(x, verbose = True)[1:5]

('The number of layers for this run is', 0)


Variable containing:
 2.7171
-0.5757
-1.1496
-0.6636
[torch.FloatTensor of size 4x1]

In [6]:
model(x, verbose = True)[1:5]

('The number of layers for this run is', 1)


Variable containing:
 2.7784
-0.5659
-1.1524
-0.6715
[torch.FloatTensor of size 4x1]

In [7]:
model(x, verbose = True)[1:5]

('The number of layers for this run is', 3)


Variable containing:
 2.7219
-0.6037
-1.1652
-0.7280
[torch.FloatTensor of size 4x1]

In [8]:
model(x, verbose = True)[1:5]

('The number of layers for this run is', 0)


Variable containing:
 2.7171
-0.5757
-1.1496
-0.6636
[torch.FloatTensor of size 4x1]

In [9]:
model(x, verbose = True)[1:5]

('The number of layers for this run is', 0)


Variable containing:
 2.7171
-0.5757
-1.1496
-0.6636
[torch.FloatTensor of size 4x1]

So what's the target?

In [10]:
y[1:5]

Variable containing:
 2.7307
-0.5801
-1.1544
-0.6970
[torch.FloatTensor of size 4x1]